In [1]:
import pandas as pd
import requests
from zipfile import ZipFile
import requests
from io import BytesIO
from datawrapper import Datawrapper
import os

In [5]:
r = requests.get("https://www150.statcan.gc.ca/n1/en/tbl/csv/18100004-eng.zip?st=G8sPC-61")
files = ZipFile(BytesIO(r.content))
file = files.open(files.namelist()[0])
raw = pd.read_csv(file, encoding="utf-8")

c:\Users\dexmc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (11,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
raw

,REF_DATE,GEO,DGUID,Products and product groups,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1914-01,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.200,6.0,NaN,NaN,NaN,1
1,1914-01,Canada,2016A000011124,All-items (1992=100),1992=100,7,units,0,v41713403,2.309,7.2,NaN,NaN,t,1
2,1914-01,Canada,2016A000011124,Goods and services,2002=100,17,units,0,v41691221,2.273,6.0,NaN,NaN,t,1
3,1914-02,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.200,6.0,NaN,NaN,NaN,1
4,1914-02,Canada,2016A000011124,All-items (1992=100),1992=100,7,units,0,v41713403,2.309,7.2,NaN,NaN,t,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049162,2022-04,"Yellowknife, Northwest Territories",2011A00056106023,Durable goods,2002=100,17,units,0,v41692831,30.275,111.9,NaN,NaN,NaN,1
1049163,2022-04,"Yellowknife, Northwest Territories",2011A00056106023,Semi-durable goods,2002=100,17,units,0,v41692832,30.276,108.0,NaN,NaN,NaN,1
1049164,2022-04,"Yellowknife, Northwest Territories",2011A00056106023,Non-durable goods,2002=100,17,units,0,v41692833,30.277,176.6,NaN,NaN,NaN,1
1049165,2022-04,"Yellowknife, Northwest Territories",2011A00056106023,Services,2002=100,17,units,0,v41692834,30.282,151.6,NaN,NaN,NaN,1


### Individual charts

In [8]:
provinces = ["British Columbia", "Newfoundland and Labrador", "Alberta", "Saskatchewan", "Manitoba", "Ontario", "Quebec", "New Brunswick", "Nova Scotia", "Prince Edward Island"]

In [11]:
gas = raw.loc[(raw["Products and product groups"] == "Gasoline") & (raw["GEO"].isin(provinces)) & (raw["REF_DATE"].gt("1980-01-01"))].pivot(values="VALUE", index="REF_DATE", columns="GEO")

gas

GEO,Alberta,British Columbia,Manitoba,New Brunswick,Newfoundland and Labrador,Nova Scotia,Ontario,Prince Edward Island,Quebec,Saskatchewan
REF_DATE,,,,,,,,,,
1980-02,30.7,31.8,36.4,34.0,36.6,34.4,35.9,39.0,35.2,34.9
1980-03,31.9,32.8,37.5,35.1,37.4,36.0,36.9,40.6,36.4,36.0
1980-04,32.0,33.7,37.7,35.1,37.7,36.1,37.1,40.8,36.4,36.1
1980-05,32.5,34.3,38.1,35.7,38.4,36.6,37.6,41.6,37.0,36.5
1980-06,32.5,34.6,38.0,35.7,38.5,36.6,37.6,42.1,37.0,36.5
...,...,...,...,...,...,...,...,...,...,...
2021-12,209.2,215.4,201.0,187.7,196.5,176.1,201.0,200.3,203.6,183.3
2022-01,210.6,231.1,212.7,191.7,206.1,190.4,212.8,213.4,210.8,190.5
2022-02,221.8,240.2,225.8,206.3,221.1,207.0,229.6,230.2,226.5,203.5


In [15]:
try:
    with open('./auth.txt', 'r') as f:
        DW_AUTH_TOKEN = f.read().strip()    
except:
    DW_AUTH_TOKEN = os.environ['DW_AUTH_TOKEN']
    
dw = Datawrapper(access_token=DW_AUTH_TOKEN)

KeyError: 'DW_AUTH_TOKEN'

In [ ]:
for prov in provinces:
    df = gas[prov]
    
    display(df)

In [ ]:




dw.create_chart(data=df)